# Brain Tumor Detection

We start by importing the necessary libraries. We'll use Keras with TensorFlow Backend for our CNN and VGG16 as the pre trained model. 

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import SGD
from keras.applications import VGG16
import matplotlib.pyplot as plt ## to plot accuracy and loss graphs 
%matplotlib inline

### Data Augmentation

As we have very less training data there is a high chance of overfitting. To overcome this, we use data augmentation which is done by the ImageDataGenerator present in Keras.

In [ ]:
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, fill_mode='nearest')
## vertical_flip is not needed as Brain MRI's have the same vertical orientation.
valid_gen = ImageDataGenerator(rescale=1./255)

Now we use the flow_from_directory method to load the images from their respective paths. We also specify our target_size and the batch_size.

In [ ]:
train = train_gen.flow_from_directory('/path/to/training/images', target_size = (160,160), batch_size=10, 
                                      class_mode='binary')
valid = valid_gen.flow_from_directory('/path/to/validation/images', target_size = (160,160), batch_size=10, 
                                      class_mode='binary')

### CNN Model

We create our Sequential Model. The model consists of 2 convolutional blocks followed by Flatten and a fully-connected layer with Dropout to reduce overfitting. 

In [3]:
model = Sequential()

model.add(Conv2D(8, 3, activation='relu', input_shape=(160,160,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(16, 3, activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid')) ## Output layer
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 158, 158, 8)       224       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 79, 79, 8)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 77, 77, 16)        1168      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 38, 38, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 23104)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               5914880   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)              

Now we compile the model. I use SGD with momentum as optimizer and the learning rate is kept very small. 

In [ ]:
sgd1 = SGD(lr=9e-5, decay=1e-6, momentum=0.99)
model.compile(optimizer=sgd1, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train, steps_per_epoch=22, epochs=50, validation_data=valid, validation_steps=4) 

In [ ]:
## Plotting accuracy graph
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('Accuracy')
plt.xlabel('epochs')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
## Plotting loss graph
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

### Pre Trained Model

I use VGG16 as the base model. Setting include_top=False ensures that the fully-connected layers of the base model are not used for training. 
Then we freeze all the weights of the layers in the base model. 

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(160,160,3))

for layer in base_model.layers:
        layer.trainable = False ##freezing the layers

Here we add our custom fully-connected layers on top of the base model. In step 1 the weights of these layers will be updated. 2 fully-connected layers are used with Dropout between them to avoid overfitting.

In [5]:
model_final = Sequential()
model_final.add(base_model)

model_final.add(Flatten())

model_final.add(Dense(1024, activation='relu'))
model_final.add(Dropout(0.4))
model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(1, activation='sigmoid'))
model_final.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 5, 5, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              13108224  
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 28,085,569
Trainable params: 13,370,881
Non-trainable params: 14,714,688
_________________________________

Now we compile our pre trained model. Again we use SGD with momentum as optimizer with a small learning rate. This time we train only for 25 epochs. Only the fully-connected layers will be trained in this step.

In [ ]:
## Step 1
sgd2 = SGD(lr=1e-4, decay=1e-6, momentum=0.9)
model_final.compile(optimizer=sgd2, loss='binary_crossentropy', metrics=['accuracy'])
model_final.fit_generator(train, steps_per_epoch=22, epochs=25, validation_data=valid, validation_steps=4)

Now step 2 begins. We first reset our ImageDataGenerators and the unfreeze the final convolutional block of the base model. 

In [ ]:
train.reset()
valid.reset()
 
for layer in base_model.layers[15:]:
    layer.trainable = True ## unfreezing the last convolutional block

Again we compile the model using the same optimizer in step 1. This time the layers present in the last convolutional block of VGG16 will also be trained along with the fully-connected layers. We train the model for 25 epochs.

In [ ]:
## Step 2
model_final.compile(optimizer=sgd2, loss='binary_crossentropy', metrics=['accuracy'])
hist = model_final.fit_generator(train, steps_per_epoch=22, epochs=25, validation_data=valid, validation_steps=4)

In [ ]:
## Plotting accuracy graph
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.ylabel('Accuracy')
plt.xlabel('epochs')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
## Plotting loss graph
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()